## Install

In [1]:
pip install tensorflow_hub transformers tensorflow python-dotenv pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


## Import Necessary Libraries

In [2]:
import os
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
from transformers import BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity

## Load Data

In [3]:
load_dotenv()  # Load environment variables from .env file
jwt_token = os.getenv('JWT_TOKEN')

headers = {'Authorization': f'Bearer {jwt_token}'}

api_urls = {
    'products': 'http://161.97.109.65:3000/api/products'
}

def fetch_data(url, headers):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raises an HTTPError for bad requests
        data = pd.DataFrame(response.json())
        print(f"Data successfully fetched from {url}")
        print(data.head())  # Display the first few rows of the DataFrame
        return data
    except requests.RequestException as e:
        print(f'Failed to fetch data from {url}: {str(e)}')
        return pd.DataFrame()

# Fetch data from APIs
products = fetch_data(api_urls['products'], headers)

# Check if   data was fetched successfully
if not products.empty:
    print("All data fetched successfully.")
else:
    print("Data fetching failed, check errors and retry.")
    # Optionally, add logic to halt further processing if data is crucial

Data successfully fetched from http://161.97.109.65:3000/api/products
                        _id category     price  \
0  6667ef73b3e75416b2fa7e33     Meja  155000.0   
1  6667ef73b3e75416b2fa7e34     Meja  124000.0   
2  6667ef73b3e75416b2fa7e35     Meja  107000.0   
3  6667ef73b3e75416b2fa7e36     Meja   99500.0   
4  6667ef73b3e75416b2fa7e37     Meja  446000.0   

                                                name  \
0  Damaindah Meja Belajar Kayu Set Kursi / Meja B...   
1  Homedoki Meja / Meja Makan / Meja Komputer / M...   
2  Sakula Meja kantor meja kerja Meja Komputer Pe...   
3  Meja Portable Stand Laptop Meja Laptop Standin...   
4  PiPi Furniture Meja Gaming / Meja komputer / M...   

                   sellerId  \
0  6665e9847aa0dfec0ad43b26   
1  6665e9847aa0dfec0ad43b26   
2  6665e9847aa0dfec0ad43b26   
3  6665e9847aa0dfec0ad43b26   
4  6665e9847aa0dfec0ad43b26   

                                        productImage  __v  
0  [https://storage.googleapis.com/kelas-app-

# Data Preprocessing

In [6]:
print(products.head())

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
def tokenize_data(texts, tokenizer, max_length=128):
    return tokenizer(
        texts.tolist(),
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='tf'
    )

# Assuming the data contains a column named 'name'
tokenized_data = tokenize_data(products['name'], tokenizer)
input_ids = tokenized_data['input_ids']
attention_masks = tokenized_data['attention_mask']

# Convert tensors to numpy arrays
input_ids_np = input_ids.numpy()
attention_masks_np = attention_masks.numpy()

# Create dummy binary labels for demonstration purposes
# Example: Assign a label based on a condition, here assuming 'Meja' category as 1 and others as 0
labels = (products['category'] == 'Category1').astype(int).values  # Example binary labels based on category

# Split the data
train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(input_ids_np, labels, test_size=0.2, random_state=42)
train_attention_masks, val_attention_masks = train_test_split(attention_masks_np, test_size=0.2, random_state=42)

# Check the shapes of the splits to ensure correctness
print(f"Train input IDs shape: {train_input_ids.shape}")
print(f"Validation input IDs shape: {val_input_ids.shape}")
print(f"Train attention masks shape: {train_attention_masks.shape}")
print(f"Validation attention masks shape: {val_attention_masks.shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Validation labels shape: {val_labels.shape}")

# Combine product name and category for Universal Sentence Encoder
titles = products['name'].tolist()
labels = products['category'].tolist()
combined_text = [f"{label} {title}" for label, title in zip(labels, titles)]

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Generate embeddings for the combined text
embeddings = embed(combined_text)

# Check the shape of the embeddings
print(embeddings.shape)

                        _id category     price  \
0  6667ef73b3e75416b2fa7e33     Meja  155000.0   
1  6667ef73b3e75416b2fa7e34     Meja  124000.0   
2  6667ef73b3e75416b2fa7e35     Meja  107000.0   
3  6667ef73b3e75416b2fa7e36     Meja   99500.0   
4  6667ef73b3e75416b2fa7e37     Meja  446000.0   

                                                name  \
0  Damaindah Meja Belajar Kayu Set Kursi / Meja B...   
1  Homedoki Meja / Meja Makan / Meja Komputer / M...   
2  Sakula Meja kantor meja kerja Meja Komputer Pe...   
3  Meja Portable Stand Laptop Meja Laptop Standin...   
4  PiPi Furniture Meja Gaming / Meja komputer / M...   

                   sellerId  \
0  6665e9847aa0dfec0ad43b26   
1  6665e9847aa0dfec0ad43b26   
2  6665e9847aa0dfec0ad43b26   
3  6665e9847aa0dfec0ad43b26   
4  6665e9847aa0dfec0ad43b26   

                                        productImage  __v  
0  [https://storage.googleapis.com/kelas-app-test...    0  
1  [https://storage.googleapis.com/kelas-app-test...   

# Create Model

In [7]:
# Define the search function
def semantic_search(query, embeddings, data, top_k):
    # Generate the embedding for the query
    query_embedding = embed([query])

    # Calculate cosine similarities
    similarities = cosine_similarity(query_embedding, embeddings).flatten()

    # Get the top_k products
    top_k_indices = np.argsort(similarities)[-top_k:][::-1]  # Corrected this line to use argsort
    if len(top_k_indices) == 0:
        return None  # Return None if no results found
    results = data.iloc[top_k_indices]
    return results

# Try Model

In [11]:
# Example usage
query = "kasur"
results = semantic_search(query, embeddings, products, top_k=10)

print(results)

                          _id category     price  \
345  6667ef73b3e75416b2fa7f8c    Kasur  185000.0   
423  6667ef73b3e75416b2fa7fda    Kasur  226320.0   
391  6667ef73b3e75416b2fa7fba    Kasur   61281.0   
324  6667ef73b3e75416b2fa7f77    Kasur   55900.0   
384  6667ef73b3e75416b2fa7fb3    Kasur   64000.0   
340  6667ef73b3e75416b2fa7f87    Kasur   90000.0   
295  6667ef73b3e75416b2fa7f5a    Kasur  498000.0   
356  6667ef73b3e75416b2fa7f97    Kasur  224000.0   
317  6667ef73b3e75416b2fa7f70    Kasur   98000.0   
381  6667ef73b3e75416b2fa7fb0    Kasur  149000.0   

                                                  name  \
345                                       kasur dewasa   
423  Kasur Lantai MOLLORCA 100cm/kasurlipat/kasurva...   
391       Kasur lipat matras kasur lantai (90x170x5cm)   
324               KASUR LIPAT 90x170x5cm...SUPER MURAH   
384              Kasur Lantai Palembang Empuk Termurah   
340  kasur palembang, kasur kapuk, kasur lantai, ka...   
295      Kasur Spring